In [ ]:
D:\IntroToDataMining-CS412\final project\benchmark\dataset0

In [ ]:
E = ['A', 'T', 'C', 'G']

In [428]:
import random
import numpy as np
import math

### Step 2: Implementing the motif-finder
Write a program that will read the “sequences.fa” file and “motiflength.txt” file in a
data set and find a motif (position weight matrix) of length given by the
“motiflength.txt” file. How you find the motif (i.e., the algorithm) is entirely up to
you. Needless to say, your program should not “look at” motif.txt or sites.txt.
The program should produce two output files for the data set:  
1) “predictedmotif.txt”, in the same format as “motif.txt” from the data set.  
2) “predictedsites.txt”, in the same format as “sites.txt” from the data set. This file
will have your program’s prediction of one site per sequence. 

## Reading Files

In [117]:
# fa files
fa_in = open("D:/IntroToDataMining-CS412/final project/benchmark/dataset0/sequences.fa","r")
fa_Seq = []# define a list to store the sequences
fa_Num = 0
for line in fa_in.readlines():# read FASTA format line by line
    line = line.rstrip()# trim the line
    fa_Num = fa_Num + 1
    fa_Seq.append(line)
fa_Seq

['GAGACCGTCACGTATGGGGCCGGGAATATGGTCACAGGGACAAGGGTTAGCCCCAGGGGCTCTGCCGTTAAAATCACTCCTAAGTAAGGGAAATTGACCGACAACAAATACTGCACGACTTCGCAAGAGATCAATTCTAGCGAGCCTCTACCAGAGACGTTGCGTTCCGGATGCCGTCGTAGGCCACGATTCATAACCGGGGACTCCCGACAATGCTGAAACTCTAATAGAGGGCGATTGTTGGACGTATTCACGTTACTAGAGAGGGCTCCCTGGAGTATGCACGTAGTACAGCCAGGCAGGTGTGATCCGCGGTGTAATGAGAGATTTAAGCCTTATGCTGTCGTCAAGTATTTTGTATCAGGCCGGCCACATCCGCCACCCGTACTACAATACATATCCGGTGGTTACGATTATTTTATACCAAGCGAGATCAAAGTTGGGCGAGCAGTTCGCATGGCTATACGTCGTATGGAGAGAACTGGGGGACTCCACTTCTG',
 'TGGCACGGAGGACCCACCTACTGGGCCAAAAAGATTGCAATCGGAGACCGTTGCACATTGTATCTACTTGCGGTCGTTCATGGGCTAATGTCCTCAAGTGAGTCCGCAAGGTGGGCACTGCTGTGTCACGAGCAACCTCAAAAGTGATAATGCATTTCGGCCACTGACAACGCGTCTGGTCCCGTTACGCGTACGCGATATATTCAACACCATGGCGCTATCGACTGTTAGTAGCCCAGAATGTGCCGCGTATTCTATGTAACATATTCGCCGTCCTCTCTGGGCGGAGACAGTCTTCCCCGTTTAGGGTGCCTTAGTCACCGGCTTCGAACCAAAGATCAGGTTTGTAACAAGGGACCGGCTTTCCAAAAACTTACTTGATGCAGGCGGGCGGTCGCCCTTTGAAACATCTAAACTCTTCATGCCAAGAGCTACAGGAGACGTGTCACTACCGCATTGTACGCGATCGCGAAATCAGAGAAATATGTCCCGG

In [13]:
# motif length motiflength.txt
motiflen_in = open("D:/IntroToDataMining-CS412/final project/benchmark/dataset0/motiflength.txt","r")
motif_len = int(motiflen_in.readline())
motif_len

8

## Gibbs sampler

#### Step1: Random initialize site

In [97]:
# max site for the sequence is
# index: len(fa_Seq[0])-motif_len
sites = [random.randint(0, (len(fa_Seq[0])-motif_len)) for i in range(fa_Num)] 
k_mers = []
for i in range(fa_Num):
    k_mers.append(list(fa_Seq[i][sites[i]:sites[i]+motif_len]))
k_mers

[['A', 'C', 'G', 'T', 'T', 'A', 'C', 'T'],
 ['T', 'G', 'G', 'G', 'C', 'C', 'A', 'A'],
 ['T', 'G', 'A', 'C', 'A', 'T', 'G', 'A'],
 ['C', 'C', 'C', 'G', 'T', 'G', 'A', 'C'],
 ['G', 'A', 'T', 'C', 'C', 'A', 'C', 'A'],
 ['G', 'G', 'T', 'T', 'G', 'T', 'T', 'A'],
 ['G', 'C', 'T', 'C', 'T', 'A', 'A', 'A'],
 ['G', 'G', 'T', 'G', 'G', 'G', 'T', 'T'],
 ['A', 'T', 'T', 'C', 'C', 'G', 'C', 'A'],
 ['C', 'C', 'G', 'A', 'A', 'T', 'A', 'T']]

#### Step2: Random ignore one sequence

In [100]:
hide_index = random.randint(0,fa_Num-1)
hide_index

6

#### Step3: Generate init motif_matrix

In [1]:
temp_motif_matrix = k_mers[:hide_index]+k_mers[hide_index+1:]
temp_motif_matrix

NameError: name 'k_mers' is not defined

#### Step4: Generate psudo_count_matrix

In [108]:
psudo_count_matrix = [[1 for i in range(motif_len)] for j in range(4)]
for i in range(motif_len):
    for j in range(fa_Num-1):
        if temp_motif_matrix[j][i] == 'A':
            psudo_count_matrix[0][i] += 1
        elif temp_motif_matrix[j][i] == 'T':
            psudo_count_matrix[1][i] += 1
        elif temp_motif_matrix[j][i] == 'C':
            psudo_count_matrix[2][i] += 1
        elif temp_motif_matrix[j][i] == 'G':
            psudo_count_matrix[3][i] += 1
psudo_count_matrix = np.array(psudo_count_matrix) #pro_matrixpsudo_count_matrix
psudo_count_matrix

array([[3, 2, 2, 2, 3, 3, 4, 6],
       [3, 2, 5, 3, 3, 4, 3, 4],
       [3, 4, 2, 4, 4, 2, 4, 2],
       [4, 5, 4, 4, 3, 4, 2, 1]])

#### Step5: Generate pro_matrix

In [114]:
pro_matrix = psudo_count_matrix/(fa_Num-1+4)
print(pro_matrix)

[[0.23076923 0.15384615 0.15384615 0.15384615 0.23076923 0.23076923
  0.30769231 0.46153846]
 [0.23076923 0.15384615 0.38461538 0.23076923 0.23076923 0.30769231
  0.23076923 0.30769231]
 [0.23076923 0.30769231 0.15384615 0.30769231 0.30769231 0.15384615
  0.30769231 0.15384615]
 [0.30769231 0.38461538 0.30769231 0.30769231 0.23076923 0.30769231
  0.15384615 0.07692308]]


#### Step6: Probabilities of all k-mers in hiden String

In [130]:
# E = ['A', 'T', 'C', 'G']
def generate_prob(pro_matrix,k_mer): #k_mer 'CAAATCCC'
    '''
    Calculate the probability of current k_mer
    '''
    dic = {'A':0,'T':1,'C':2,'G':3}
    p = 1
    for i in range(len(k_mer)):
        p = p * pro_matrix[dic[k_mer[i]]][i]
    return p

#### Step7: Sample a k-mer from the hidden string with probability and update motif matrix 

In [410]:
prob = []
hiden_string = fa_Seq[hide_index]
for i in  range(len(fa_Seq[0])-motif_len+1):
    temp_k_mer = hiden_string[i:i+motif_len]
    prob.append(generate_prob(pro_matrix,temp_k_mer))
updated_site = np.random.choice(len(fa_Seq[0])-motif_len+1,1,prob)[0]
updated_site

8

In [413]:
sites[hide_index] = updated_site
sites

[252, 21, 314, 103, 143, 85, 8, 5, 269, 296]

In [418]:
temp_k_mers = k_mers
temp_k_mers[hide_index] = list(fa_Seq[hide_index][sites[hide_index]:sites[hide_index]+motif_len])
temp_k_mers

[['A', 'C', 'G', 'T', 'T', 'A', 'C', 'T'],
 ['T', 'G', 'G', 'G', 'C', 'C', 'A', 'A'],
 ['T', 'G', 'A', 'C', 'A', 'T', 'G', 'A'],
 ['C', 'C', 'C', 'G', 'T', 'G', 'A', 'C'],
 ['G', 'A', 'T', 'C', 'C', 'A', 'C', 'A'],
 ['G', 'G', 'T', 'T', 'G', 'T', 'T', 'A'],
 ['T', 'G', 'G', 'T', 'G', 'T', 'C', 'A'],
 ['G', 'G', 'T', 'G', 'G', 'G', 'T', 'T'],
 ['A', 'T', 'T', 'C', 'C', 'G', 'C', 'A'],
 ['C', 'C', 'G', 'A', 'A', 'T', 'A', 'T']]

#### Step8: Calculate F to decide whethter to update or not
F shows whether the new motif is closer to the optimal, so we update site and k-mers only if F increases

In [442]:
def F_given_model(temp_k_mers,background_p):
    '''
    Calculate F
    temp_k_mers : 
    [['A', 'C', 'G', 'T', 'T', 'A', 'C', 'T'],
     ['T', 'G', 'G', 'G', 'C', 'C', 'A', 'A'],
     ['T', 'G', 'A', 'C', 'A', 'T', 'G', 'A'],
     ['C', 'C', 'C', 'G', 'T', 'G', 'A', 'C'],
     ['G', 'A', 'T', 'C', 'C', 'A', 'C', 'A'],
     ['G', 'G', 'T', 'T', 'G', 'T', 'T', 'A'],
     ['T', 'G', 'G', 'T', 'G', 'T', 'C', 'A'],
     ['G', 'G', 'T', 'G', 'G', 'G', 'T', 'T'],
     ['A', 'T', 'T', 'C', 'C', 'G', 'C', 'A'],
     ['C', 'C', 'G', 'A', 'A', 'T', 'A', 'T']]
    background_p : [0.2496, 0.2492, 0.2516, 0.2496]
    '''
    dic = {'A':0,'T':1,'C':2,'G':3}
    sigma = ['A','T','C','G']
    
    #motif_count_matrix
    count_matrix = [[0 for i in range(motif_len)] for j in range(4)]
    for i in range(motif_len):
        for j in range(fa_Num):
            if temp_k_mers[j][i] == 'A':
                count_matrix[0][i] += 1
            elif temp_k_mers[j][i] == 'T':
                count_matrix[1][i] += 1
            elif temp_k_mers[j][i] == 'C':
                count_matrix[2][i] += 1
            elif temp_k_mers[j][i] == 'G':
                count_matrix[3][i] += 1
    count_matrix = np.array(count_matrix)
    '''
    [[3, 2, 2, 2, 3, 3, 4, 6],
     [3, 2, 5, 3, 3, 4, 3, 4],
     [3, 4, 2, 4, 4, 2, 4, 2],
     [4, 5, 4, 4, 3, 4, 2, 1]]
    '''
    
    #motif_probability_matrix
    pro_matrix = count_matrix/fa_Num
    '''
    [[0.23076923 0.15384615 0.15384615 0.15384615 0.23076923 0.23076923
      0.30769231 0.46153846]
     [0.23076923 0.15384615 0.38461538 0.23076923 0.23076923 0.30769231
      0.23076923 0.30769231]
     [0.23076923 0.30769231 0.15384615 0.30769231 0.30769231 0.15384615
      0.30769231 0.15384615]
     [0.30769231 0.38461538 0.30769231 0.30769231 0.23076923 0.30769231
      0.15384615 0.07692308]]
    '''
    #print(count_matrix)
    F = 0
    for i in range(len(temp_k_mers[0])):
        for j in range(4):
            if count_matrix[j][i] == 0:
                F += 0
            else:
                F += count_matrix[j][i]*math.log2(pro_matrix[j][i]/background_p[j])
    return F

In [427]:
# Calculate background probability
dic = {'A':0,'T':1,'C':2,'G':3}
background_c = [0 for i in range(4)]
background_c = np.array(background_c)
for i in fa_Seq:
    for j in i:
        background_c[dic[j]] +=1
background_p = background_c/sum(background_c)
background_p

array([0.2496, 0.2492, 0.2516, 0.2496])

In [443]:
F_given_model(temp_k_mers,[0.2496, 0.2492, 0.2516, 0.2496])

17.69739653068369